# ABM

## Imports / Data Conversion

In [2]:
import time, enum, math
import numpy as np
import pandas as pd
import pylab as plt
from mesa import Agent, Model
from mesa.time import SimultaneousActivation, RandomActivation
from mesa.space import NetworkGrid
from mesa.datacollection import DataCollector
from networkx.algorithms.shortest_paths.generic import has_path
import networkx as nx
import panel as pn
import panel.widgets as pnw
import random
from tqdm import tqdm, trange
from time import sleep

data_path = '../' #set to wherever the data files are, will be used on every input

In [3]:
ports = pd.read_csv((data_path +'ports.csv'))


origin = pd.read_csv((data_path + 'origin_ports.csv'))
clean_distances = pd.read_csv((data_path + 'clean_distances.csv'))
# distances = distances_df[["prev_port", "next_port", "distance"]]
# distances.astype({'prev_port':'int64', 'next_port':'int64'}).dtypes

In [4]:
route_blockage_dov = pd.read_csv((data_path + 'route_blockages_dov.csv'))
route_blockage_gib = pd.read_csv((data_path + 'route_blockages_gib.csv'))
route_blockage_horm = pd.read_csv((data_path + 'route_blockages_horm.csv'))
route_blockage_mal = pd.read_csv((data_path + 'route_blockages_mal.csv'))
route_blockage_pan = pd.read_csv((data_path + 'route_blockages_pan.csv'))
route_blockage_suez = pd.read_csv((data_path + 'route_blockages_suez.csv'))
route_blockage_total = pd.read_csv((data_path + 'route_blockages_total.csv'))

In [5]:
pruning_files = [route_blockage_dov, route_blockage_gib, route_blockage_horm, route_blockage_mal, route_blockage_pan, route_blockage_suez, route_blockage_total]

In [5]:
def Cut_Graph(G, route_blockages):
        return_G = G.copy()
        fails = 0
        for index in range(len(route_blockages)):
            try:
                return_G.remove_edge(route_blockages.iloc[index]['prev_port'],route_blockages.iloc[index]['next_port'])
            except:
                fails += 1
        print("Failed {} times".format(fails))
        return return_G

## Playground 

In [6]:
G = nx.from_pandas_edgelist(clean_distances, "prev_port", "next_port",edge_attr= "distance",create_using=nx.Graph())

In [7]:
route_blockage_dov.head()

,Unnamed: 0,prev_port,next_port
0,232,641,669
1,239,641,2742
2,240,641,2787
3,246,641,3367
4,248,641,3509


In [28]:
nx.dijkstra_path(G, 641, 2508, weight='distance')
# nx.dijkstra_path_length(G, 641, 669, weight='distance')

[641, 669, 2508]

In [29]:
nx.dijkstra_path(G_Dov, 641, 2508, weight='distance')
# nx.dijkstra_path_length(G_Dov,  641, 669, weight='distance')

[641, 27373, 2560, 2789, 2091, 2508]

In [12]:
G_Dov = Cut_Graph(G, route_blockage_dov)

Failed 248 times


In [23]:
results = []
Success = 0
Fail = 0
Same_distance = 0
for i in trange(3000):
    position = random.choice(list(G.nodes()))
    destination = random.choice(list(G.nodes()))
    try:
        init_route = nx.dijkstra_path(G, position, destination, weight='distance')
        init_dist = nx.dijkstra_path_length(G, position, destination, weight='distance')
        Total_route = nx.dijkstra_path(G_Dov, position, destination, weight='distance')
        Total_dist = nx.dijkstra_path_length(G_Dov, position, destination, weight='distance')
        if init_route != Total_route:
            result = [position, destination]
            results.append(result)
            Success += 1
        else:
            Same_distance += 1
    except :
        Fail += 1


100%|██████████| 3000/3000 [04:49<00:00, 10.37it/s]


In [24]:
print(Success, Same_distance, Fail)

638 2201 161


In [25]:
print(638/3000, 2201/3000, 161/3000)

0.21266666666666667 0.7336666666666667 0.05366666666666667


In [62]:
# results = []
# Success = 0
# Fail = 0
# Same_distance = 0
# for i in trange(len(list(G.nodes()))):
#     position = list(G.nodes())[i]
#     for j in range(len(list(G.nodes()))):
#         destination = list(G.nodes())[j]
#         try:
#             init_route = nx.dijkstra_path(G, position, destination, weight='distance')
#             init_dist = nx.dijkstra_path_length(G, position, destination, weight='distance')
#             Total_route = nx.dijkstra_path(G_Total, position, destination, weight='distance')
#             Total_dist = nx.dijkstra_path_length(G_Total, position, destination, weight='distance')
#             if init_route != Total_route:
#                 result = [position, destination]
#                 results.append(result)
#                 Success += 1
#             else:
#                 Same_distance += 1
#         except :
#             Fail += 1

        

  0%|          | 0/2912 [00:00<?, ?it/s]

In [12]:
init_route = nx.dijkstra_path(G, position, destination, weight='distance')
init_dist = nx.dijkstra_path_length(G, position, destination, weight='distance')
Total_route = nx.dijkstra_path(G_Suez, position, destination, weight='distance')
Total_dist = nx.dijkstra_path_length(G_Suez, position, destination, weight='distance')

In [14]:
init_dist

6063.350807988509

In [ ]:
# def next_dest(sample):
#     try:

#         overall_distance = list()
#         itinerary = list()
#         for i in range(len(sample)):
#             distance = dict()
#             for i in range(1,len(sample)):
#                 distance[sample[i]] = nx.dijkstra_path_length(G, sample[0] ,sample[i], weight='distance')
#             next_stop = min(distance, key=distance.get)
#             itinerary.append(nx.dijkstra_path(G, origin, next_stop, weight = 'distance'))
#             overall_distance.append(next_stop)
#             sample.pop(0)
#             ind = sample.index(next_stop)
#             sample.pop(ind)
#             sample.insert(0, next_stop)
#     except:
#         pass
#     flat_route = []
#     for sublist in itinerary:
#         for port in sublist:
#             flat_route.append(port)
        
#     return flat_route, sum(overall_distance)


In [ ]:
pruning_files = [route_blockage_dov, route_blockage_gib, route_blockage_horm, route_blockage_mal, route_blockage_pan, route_blockage_suez, route_blockage_total]

## Model

In [13]:
class ShippingNetwork(Model):
    def __init__(self, distances, major_ports, pruning_files, S=1000):
        self.major_ports = major_ports
        self.num_ships = S
        self.distances = distances
        self.schedule = SimultaneousActivation(self)
        self.running = True
        self.Ships = []
        self.pruning_files = pruning_files

        #Build Network without closures
        self.G = nx.from_pandas_edgelist(distances, "prev_port", "next_port", ["distance"], create_using=nx.Graph())    
        #Define Mesa Grid as the just created Network to allow for shipping only in routes
        self.grid = NetworkGrid(self.G) 
    
        #Build alternate Networks (with closures in place)
        self.G_Dov = self.Cut_Graph(self.G, self.pruning_files[0])
        self.G_Gib = self.Cut_Graph(self.G, self.pruning_files[1])
        self.G_Horm = self.Cut_Graph(self.G, self.pruning_files[2])
        self.G_Mal = self.Cut_Graph(self.G, self.pruning_files[3])
        self.G_Pan = self.Cut_Graph(self.G, self.pruning_files[4])
        self.G_Suez = self.Cut_Graph(self.G, self.pruning_files[5])
        self.G_Total = self.Cut_Graph(self.G, self.pruning_files[6])
  




        #create agents   
        for i in tqdm(range(self.num_ships), desc="Placing Ships"):
            # start_port = np.random.choice(self.major_ports["Ref"], size=None, replace=True,  p=self.major_ports["PROB"])
            # destin_port = random.sample(self.G.nodes, k=1)[0]  # Sample the destination 
            start_port = 641
            destin_port = 2508
            #We sample the origin port from a list of the 50 biggest ports world, with the prob = TAU of the port / TAU of all origin ports for 2017
            a = Ship(i+1, self, self.G, start_port, destin_port, major_ports)
            self.schedule.add(a)
            self.Ships.append(a) #append to list of ships
            self.grid.place_agent(a, start_port) #place agent on origin node

        self.datacollector = DataCollector(
            model_reporters={"Graph":"G"},
            agent_reporters={"Position": "position", "Destination":"destination", "State":"state", "Itinerary":"itinerary", "Distance_Traveled":"distance_traveled", "Route":"current_route", "Route Changes":"route_chng" })
    


    def Cut_Graph(self, G, route_blockages):
        return_G = G.copy()
        for index in range(len(route_blockages)):
            try:
                return_G.remove_edge(route_blockages.iloc[index]['prev_port'],route_blockages.iloc[index]['next_port'])
            except:
                pass
        return return_G


        #create ability to remove edges mid-model
    def network_change(self, blockage):
        if blockage == "Dover":
            for a in tqdm(self.Ships):
                a.G = self.G_Dov
        elif blockage == "Gibraltar":
            for a in tqdm(self.Ships):
                a.G = self.G_Gib
        elif blockage == "Hormuz":
            for a in tqdm(self.Ships):
                a.G = self.G_Horm
        elif blockage == "Malacca":
            for a in tqdm(self.Ships):
                a.G = self.G_Mal
        elif blockage == "Panama":
            for a in tqdm(self.Ships):
                a.G = self.G_Pan
        elif blockage == "Suez":
            for a in tqdm(self.Ships):
                a.G = self.G_Suez
        elif blockage == "Total":
            for a in tqdm(self.Ships):
                a.G = self.G_Total
        else:
            for a in tqdm(self.Ships):
                a.G = self.G


    def step(self, blockage=''):
        #check network for changes
        if blockage != '' :
           network_change(blockage)

        self.schedule.step()     #Run each Agents
        self.datacollector.collect(self)




In [14]:
class Ship(Agent):
    def __init__(self, unique_id, model, G, start_port, destin_port, major_ports, s=13.0):
        super().__init__(unique_id, model)
        self.destination = destin_port
        self.state = 0 #0 for active, numbers > 0 for weeks that ships have to "wait" until arrival to port
        self.speed = s*24*1.852 #speed is given in knots, with 1 knot being 1 nautical mile per hour. Since the model works with distances in km, we convert here (1 nm = 1.852m)
        self.position = start_port
        self.distance_traveled = 0
        self.unique_id = unique_id
        self.G = G
        self.route_chng = 0
        self.routes = 0
        self.major_ports = major_ports

    
        #implement dijkstra to define shortest route
        try:
            self.init_route = nx.dijkstra_path(self.G, self.position, self.destination, weight='length')
            self.init_dist = nx.dijkstra_path_length(self.G, self.position, self.destination, weight='length')
        except:
            self.destination = random.sample(self.G.nodes, k=1)[0]
            self.init_route = nx.dijkstra_path(self.G, self.position, self.destination, weight='length')
            self.init_dist = nx.dijkstra_path_length(self.G, self.position, self.destination, weight='length')
            #include a way for capacity?
        

         #We keep a copy of the entire itinerary / distance traveled
        self.current_route, self.current_dist = self.init_route, self.init_dist  #For comparison & navigational purposes, we use current route & distance
        self.next_position = self.current_route[1]
        self.itinerary = [self.position]
        self.step_size = self.ident_distance()


    def routing(self):
        #implement dijkstra to define shortest route
        try:
            route = nx.dijkstra_path(self.G, self.position, self.destination, weight='length')
            travel_distance = nx.dijkstra_path_length(self.G, self.position, self.destination, weight='length')
        except:
            self.destination = random.sample(self.G.nodes, k=1)[0]
            route = nx.dijkstra_path(self.G, self.position, self.destination, weight='length')
            travel_distance = nx.dijkstra_path_length(self.G, self.position, self.destination, weight='length')
        #include a way for capacity?
        return route, travel_distance


    def move(self):
        self.next_position = self.current_route[1]
        self.step_size = self.ident_distance() #look up the distance between two cities 
        self.state = self.step_size / self.speed #change state to step amount
        self.current_dist = self.current_dist - self.step_size #adjust current distance minus the distance traveled in the next step
        self.model.grid.move_agent(self, self.next_position) #move the agent
        self.current_route.remove(self.current_route[0]) #remove the next step from the itinerary
        self.position = self.next_position
        if len(self.current_route) == 1:
            self.next_position = self.current_route[0] 
        else:
            self.next_position = self.current_route[1] #update current route

    def ident_distance(self): #look up the distance of the current step
        return self.G.get_edge_data(self.position, self.next_position)['distance']
    
    def reroute(self):
        # self.routes += 1
        # if self.position in self.major_ports["Ref"].values: # We re-route in such a way that ships that have either an origin port or a destination port that is a major port
        #     self.destination = random.sample(self.G.nodes, k=1)[0]
        # else:
        #     self.destination = np.random.choice(self.major_ports["Ref"], size=None, replace=True,  p=self.major_ports["PROB"])
        if self.destination == 2508:
            self.destination = 641
        else:
            self.destination = 2508

        self.current_route, self.current_dist = self.routing()
        self.state = 3

    

    def step(self):
        self.state = self.state - 1 #'move' ships by one weeks progress
        if self.state <= 0.000: #ships that are en-route to the node they are going to next do not move / perform other activities
            self.distance_traveled += self.step_size #ship has arrived at port, let's add the distance traveled to their 
           
            #add the current position to itinerary
            if self.position != self.destination: #if current stop is not the final stop
                new_route, new_distance = self.routing() #perform a new routing to compare against current routing
                
                if new_route == self.current_route: #if current routing is the same as new, just move (default case)
                    # print("default case")
                    self.move()
                    self.itinerary.append(self.position)
        
            # THIS CURRENTLY ONLY CHANGES THE ROUTE IF THE NEXT STEP IS BLOCKED
                elif new_distance > self.current_dist: #if current route is shorter than newly calculated route, check for obstructions
                    # print('reroute: current route shorter than new route')
                    if not has_path(self.G, self.position, self.next_position): 
                        self.current_route = new_route
                        self.current_dist = new_distance
                        self.route_chng += 1
                    self.move()
                    self.itinerary.append(self.position)
                
                
                else: # final option is that current route is longer than new route (think Suez reopening after a while), here, we just take the new option
                    # print('reroute: current route longer than new route')
                    self.current_route = new_route
                    self.current_dist = new_distance
                    self.route_chng += 1
                    self.move()
                    self.itinerary.append(self.position)
            
            else: #if ship is arrived at final position, get a new route, and start back
                # print('arrival')
                self.reroute()
                self.itinerary.append(self.position)
        # print("Ship: {}, Source: {}, Destination: {}, Position: {},  Next Stop {}, Time until next Stop {}".format(self.unique_id, self.itinerary[0], self.destination, self.position, self.next_position, self.state ))

    
    # def collect_time

    # def collect_costs

## Model Instances

In [11]:

model = ShippingNetwork(clean_distances, origin, pruning_files, 1)

steps = 15
for i in trange(steps, desc="Stepping Model"):
    model.step()
model.network_change('Dover')
for i in trange(steps, desc="Stepping Model after changes"):
    model.step()
print("does this execute here?")


agent_state = model.datacollector.get_agent_vars_dataframe()
agent_state

Stepping Model after changes:   7%|▋         | 1/15 [00:00<00:01,  9.23it/s]
done changing, should step again?


TypeError: 'NoneType' object is not subscriptable

In [17]:

agent_state = model.datacollector.get_agent_vars_dataframe()
agent_state


,,Position,Destination,State,Itinerary,Distance_Traveled,Route,Route Changes
Step,AgentID,,,,,,,
1,1,669,2508,0.247789,"[641, 669, 2508, 2508, 669, 641, 641, 669, 250...",143.178242,[2508],0
2,1,2508,2508,0.022111,"[641, 669, 2508, 2508, 669, 641, 641, 669, 250...",286.356483,[2508],0
3,1,2508,641,3.000000,"[641, 669, 2508, 2508, 669, 641, 641, 669, 250...",299.132941,[641],0
4,1,2508,641,2.000000,"[641, 669, 2508, 2508, 669, 641, 641, 669, 250...",299.132941,[641],0
5,1,2508,641,1.000000,"[641, 669, 2508, 2508, 669, 641, 641, 669, 250...",299.132941,[641],0
6,1,669,641,0.022111,"[641, 669, 2508, 2508, 669, 641, 641, 669, 250...",311.909398,[641],0
7,1,641,641,0.247789,"[641, 669, 2508, 2508, 669, 641, 641, 669, 250...",324.685856,[641],0
8,1,641,2508,3.000000,"[641, 669, 2508, 2508, 669, 641, 641, 669, 250...",467.864097,[2508],0
9,1,641,2508,2.000000,"[641, 669, 2508, 2508, 669, 641, 641, 669, 250...",467.864097,[2508],0


In [20]:
agent_state.loc[16]

,Position,Destination,State,Itinerary,Distance_Traveled,Route,Route Changes
AgentID,,,,,,,
1,669,641,0.022111,"[641, 669, 2508, 2508, 669, 641, 641, 669, 250...",779.773495,"[669, 641]",0


138038.67166337415